In [2]:
from utils.database_connection import DatabaseConnection
import utils.results_repository as rep
import pandas as pd
import numpy as np
from utils.column_name_utils import *
import utils.query_utils as query
import matplotlib.pyplot as plot
import utils.dataframe_utils as df_util
import utils.path_utils as path_util
import utils.math_utils as math_util
import math
import utils.latex_utils as tex
from scipy import stats
import utils.time_series_utils as ts_util
import json
import utils.measurement_utils as m_util

In [3]:
db = DatabaseConnection()

Connected to MySQL Server version  8.0.32-24


In [4]:
measurements_used = 10000
time_series_used = 10
# energy_per_time = 1000
generate_graphs = False
show_plots = False

get_timeseries = False
get_measurements = True

In [5]:
# exp_name = "compiler"
# exp_name = "measuring_instrument"
exp_name = "cores"

In [6]:
used_dut_name = [
    'workstationtwo',
    'workstationone',
]

used_dut_name.sort()

used_dut_env = 'prod'

windows = "win32nt"
linux = "unix"

# used_os_names = [
#     windows, 
#     linux
# ]

used_test_case_names = [
    'fannkuch-redux.exe',
    'mandelbrot.exe',
    # 'idle-case.exe'
]

used_test_case_names.sort()

used_thread = [
    'multi-threaded'
]

used_thread.sort()

used_test_case_name = 'idle-case.exe'

ipg = "ipg"
lhm = "lhm"
rapl = "rapl"
clamp = "clamp"
plug = "plug"
scaphandre = "scaphandre"

windows_mi = [
    ipg, lhm, clamp, plug, scaphandre
]

linux_mi = [
    rapl, clamp, plug
]

used_mi_names = [
    clamp, ipg ##ipg, lhm, ipg, clamp, plug #, scaphandre, # plug, rapl
]

ipg_sample_rate = 100
lhm_sample_rate = 100
rapl_sample_rate = 100
clamp_sample_rate = 100
plug_sample_rate = 200
scaphandre_sample_rate = 100

used_min_temp = 0
used_max_temp = 200
used_burn_in = 1
used_allocated_cores = [
    []
]

used_allocated_cores.sort()

In [7]:
if exp_name == "compiler":
    experiment_name = "same-cpp-multi-core-micro-benchmark-different-compiler"
    experiment_number = 1
    
    used_os_names = [
        windows, 
    ]
    
    used_mi_names = [
        ipg
    ]
    
    used_compilers = [
        'clang', 'intel-one-api', 'min-gw', 'msvc'
    ]
    
    test_case_parameters = ["", "12", "16000"]

if exp_name == "measuring_instrument":
    experiment_name = "same-one-api-compiler-different-measuring-instruments-post-update-and-watt"
    experiment_number = 2
    
    used_os_names = [
        windows, 
        linux
    ]
    
    used_mi_names = [
        ipg, scaphandre, lhm, plug, rapl, clamp, rapl
        # clamp
        # ipg, clamp, plug, lhm, rapl #, lhm, scaphandre, # plug, rapl
        # lhm#, plug
        # lhm #clamp
        # rapl, plug
    ]
    
    used_compilers = [
        'intel-one-api'
    ]
    
    test_case_parameters = ["", "12", "64000"]

if exp_name == 'cores':
    experiment_name = "same-one-api-compiler-different-cores"
    experiment_number = 3
    
    used_os_names = [
        windows, 
    ]
    
    used_mi_names = [
        ipg #, clamp
    ]
    
    used_compilers = [
        'intel-one-api'
    ]
    
    used_test_case_names = [
        'nbody.exe',
        'spectral-norm.exe',
        # 'idle-case.exe'
    ]
    
    used_thread = [
        'single-threaded'
    ]
    
    used_allocated_cores = [
        ['0'], 
        ['1'], 
        ['2'],
        ['3'],
        ['4'],
        ['5'],
        ['6'],
        ['7'],
        ['8'],
        ['9'],
    ]
    
    used_os_names = [
        windows
    ]   
    
    test_case_parameters = ["", "180000000", "18000"]


used_mi_names.sort()
used_compilers.sort()

In [8]:
used_test_case_names

['nbody.exe', 'spectral-norm.exe']

### DUTs

In [9]:
used_dut_name_str = "','".join(used_dut_name)
used_os_name_str = "','".join(used_os_names)

In [10]:
dut = rep.get_devices_under_test(db)
dut.shape

(9, 4)

In [11]:
used_dut = dut.query(f"{ENV} == '{used_dut_env}' and {NAME} in ('{used_dut_name_str}') and Os in ('{used_os_name_str}')")

used_dut.shape

(2, 4)

### Test Cases

In [12]:
tc = rep.get_test_cases(db)

tc.shape

(30, 8)

In [13]:
used_test_case_name_str = "','".join(used_test_case_names)
used_compiler_str = "','".join(used_compilers)
used_thread_str = "','".join(used_thread)
used_test_case_parameters = "','".join(test_case_parameters)

used_tc = tc.query(f"{COMPILER} in ('{used_compiler_str}') and {NAME} in ('{used_test_case_name_str}') and {THREADS} in ('{used_thread_str}') and Parameter in ('{used_test_case_parameters}')")

In [14]:
used_idle_case = tc.query(f"{COMPILER} in ('{used_compiler_str}') and {NAME} == '{used_test_case_name}' and {THREADS} in ('{used_thread_str}')")

In [15]:
used_test_case_parameters

"','180000000','18000"

### Measuring Instruments

In [16]:
mi = rep.get_measuring_instruments(db)

mi.shape

(15, 3)

In [17]:
used_mi_query_list =  query.get_mi_query(
    used_mi_names,
    ipg,
    lhm,
    rapl,
    plug,
    clamp,
    scaphandre,
    ipg_sample_rate,
    lhm_sample_rate,
    rapl_sample_rate,
    plug_sample_rate,
    clamp_sample_rate,
    scaphandre_sample_rate,
)

used_mi = mi.query(" or ".join(used_mi_query_list))

### Configurations

In [18]:
used_allocated_cores_str = ",".join([str(x) for x in used_allocated_cores])
used_allocated_cores_str = used_allocated_cores_str.replace("'", '')
used_allocated_cores_str = used_allocated_cores_str.replace("[", "'[")
used_allocated_cores_str = used_allocated_cores_str.replace("]", "]'")

In [19]:
config = rep.get_configurations(db)

config.shape

(15, 5)

In [20]:
config.shape

(15, 5)

In [21]:
used_configs = config.query(f"{MIN_TEMP} == {used_min_temp} and " +
                            f"{MAX_TEMP} == {used_max_temp} and " +
                            f"{BURN_IN} == {used_burn_in} and " +
                            # f"{ALLOCATED_CORES} in ('[1]')")
                            f"{ALLOCATED_CORES} in ({used_allocated_cores_str})")
                            # f"CAST({ALLOCATED_CORES} as CHAR) = [{used_allocated_cores_str}]")
used_configs

,Id,MinTemp,MaxTemp,Burnin,AllocatedCores
5,6,0,200,1,[0]
6,7,0,200,1,[1]
7,8,0,200,1,[2]
8,9,0,200,1,[3]
9,10,0,200,1,[4]
10,11,0,200,1,[5]
11,12,0,200,1,[6]
12,13,0,200,1,[7]
13,14,0,200,1,[8]
14,15,0,200,1,[9]


### Experiment Names

In [22]:
exp = rep.get_distinct_experiments(db)

exp.shape

(7, 2)

## Used Things

In [23]:
used_dut

,Id,Name,Os,Env
0,1,workstationone,win32nt,prod
4,5,workstationtwo,win32nt,prod


In [24]:
used_tc

,Id,Name,Compiler,Language,Optimizations,TestCaseSize,Parameter,Threads
27,31,nbody.exe,intel-one-api,c++,,microbenchmarks,180000000,single-threaded
28,32,spectral-norm.exe,intel-one-api,c++,,microbenchmarks,18000,single-threaded


In [25]:
used_mi

,Id,Name,SampleRate
0,1,ipg,100


In [26]:
used_idle_case

,Id,Name,Compiler,Language,Optimizations,TestCaseSize,Parameter,Threads
29,33,idle-case.exe,intel-one-api,c++,,microbenchmarks,,single-threaded


In [27]:
used_configs

,Id,MinTemp,MaxTemp,Burnin,AllocatedCores
5,6,0,200,1,[0]
6,7,0,200,1,[1]
7,8,0,200,1,[2]
8,9,0,200,1,[3]
9,10,0,200,1,[4]
10,11,0,200,1,[5]
11,12,0,200,1,[6]
12,13,0,200,1,[7]
13,14,0,200,1,[8]
14,15,0,200,1,[9]


In [28]:
time_series = {}

if get_timeseries:
    time_series = ts_util.get_time_series(
        used_dut,
        used_mi,
        used_configs,
        used_tc,
        experiment_name,
        experiment_number,
        linux,
        windows,
        windows_mi,
        linux_mi,
        time_series_used,
        db,
        scaphandre
    )

In [29]:
# [x for x in time_series[0]["fannkuch-redux.exe.rapl.workstationone.all.intel-one-api.unix"].iterrows()]

In [30]:
for key in time_series.keys():
    print(key)
    for table in time_series[key]:
        if "clamp" in key or "plug" in key or "scaphandre" in key or 'lhm' in key:
            table.plot(x="ElapsedTime", y="CpuEnergyInJoules")
        if "ipg" in key:
            table.plot(x="ElapsedTime", y="ProcessorPowerInWatt")
        if "rapl" in key:
            table['rapl_column'] = df_util.get_rapl_column(table, "total_core")
            table.plot(x="ElapsedTime", y="rapl_column")
        plot.show()

In [31]:
used_configs.shape, used_dut.shape, used_mi.shape, used_tc.shape, used_idle_case.shape


((10, 5), (2, 4), (1, 3), (2, 8), (1, 8))

In [32]:
used_tc

,Id,Name,Compiler,Language,Optimizations,TestCaseSize,Parameter,Threads
27,31,nbody.exe,intel-one-api,c++,,microbenchmarks,180000000,single-threaded
28,32,spectral-norm.exe,intel-one-api,c++,,microbenchmarks,18000,single-threaded


In [33]:
cpu_energy_results = {}
dram_energy_results = {}
gpu_energy_results = {}
duration_results = {}

idle_cpu_consumption_results = {}

cpu_dynamic_energy_watt_consumption = {}

cpu_dynamic_energy_consumption = {}
gpu_dynamic_energy_consumption = {}
dram_dynamic_energy_consumption = {}

temperature_begin = {}
temperature_end = {}

if get_measurements:
    m_util.get_measurements(
        windows,
        windows_mi,
        linux,
        linux_mi,
        used_configs,
        used_dut,
        used_mi,
        used_tc,
        used_idle_case,
        db,
        experiment_name,
        experiment_number,
        # energy_per_time,
        scaphandre,
        cpu_energy_results,
        dram_energy_results,
        gpu_energy_results,
        duration_results,
        temperature_begin,
        temperature_end,
        cpu_dynamic_energy_consumption,
        gpu_dynamic_energy_consumption,
        dram_dynamic_energy_consumption,
        idle_cpu_consumption_results,
        used_test_case_name,
        cpu_dynamic_energy_watt_consumption
    )

Y - nbody.exe.win32nt.workstationone.[0].intel-one-api.ipg.100.31
Y - spectral-norm.exe.win32nt.workstationone.[0].intel-one-api.ipg.100.32
Y - nbody.exe.win32nt.workstationone.[1].intel-one-api.ipg.100.31
Y - spectral-norm.exe.win32nt.workstationone.[1].intel-one-api.ipg.100.32
Y - nbody.exe.win32nt.workstationone.[2].intel-one-api.ipg.100.31
Y - spectral-norm.exe.win32nt.workstationone.[2].intel-one-api.ipg.100.32
Y - nbody.exe.win32nt.workstationone.[3].intel-one-api.ipg.100.31
Y - spectral-norm.exe.win32nt.workstationone.[3].intel-one-api.ipg.100.32
Y - nbody.exe.win32nt.workstationone.[4].intel-one-api.ipg.100.31
Y - spectral-norm.exe.win32nt.workstationone.[4].intel-one-api.ipg.100.32
Y - nbody.exe.win32nt.workstationone.[5].intel-one-api.ipg.100.31
Y - spectral-norm.exe.win32nt.workstationone.[5].intel-one-api.ipg.100.32
Y - nbody.exe.win32nt.workstationone.[6].intel-one-api.ipg.100.31
Y - spectral-norm.exe.win32nt.workstationone.[6].intel-one-api.ipg.100.32
Y - nbody.exe.win32n

In [34]:
if get_measurements:
    df_util.trim_dataframes(
        measurements_used,
        dram_energy_results,
        cpu_energy_results,
        gpu_energy_results,
        duration_results,
        cpu_dynamic_energy_consumption,
        gpu_dynamic_energy_consumption,
        dram_dynamic_energy_consumption,
        idle_cpu_consumption_results,
        temperature_end,
        temperature_begin,
        cpu_dynamic_energy_watt_consumption
    )

In [35]:
def convert_to_series(df):
    for key in df.keys():
        df[key] = pd.Series(df[key])
    return df

In [36]:
if get_measurements:
    ## Energy consumption for the entire test case
    dram_energy_results = convert_to_series(dram_energy_results)
    dram_df = pd.DataFrame(dram_energy_results)
    dram_df =dram_df.reindex(columns=sorted(dram_df.columns))

    cpu_energy_results = convert_to_series(cpu_energy_results)
    cpu_df = pd.DataFrame(cpu_energy_results)
    cpu_df = cpu_df.reindex(columns=sorted(cpu_df.columns))

    gpu_energy_results = convert_to_series(gpu_energy_results)
    gpu_df = pd.DataFrame(gpu_energy_results)
    gpu_df = gpu_df.reindex(columns=sorted(gpu_df.columns))

    duration_results = convert_to_series(duration_results)
    duration_df = pd.DataFrame(duration_results)
    duration_df = duration_df.reindex(columns=sorted(duration_df.columns))

    ## Dynamic energy data frames
    dram_dynamic_energy_consumption = convert_to_series(dram_dynamic_energy_consumption)
    dram_dynamic_df = pd.DataFrame(dram_dynamic_energy_consumption)
    dram_dynamic_df = dram_dynamic_df.reindex(columns=sorted(duration_df.columns))

    cpu_dynamic_energy_consumption = convert_to_series(cpu_dynamic_energy_consumption)
    cpu_dynamic_df = pd.DataFrame(cpu_dynamic_energy_consumption)
    cpu_dynamic_df = cpu_dynamic_df.reindex(columns=sorted(cpu_dynamic_df.columns))
    
    cpu_dynamic_energy_watt_consumption = convert_to_series(cpu_dynamic_energy_watt_consumption)
    cpu_dynamic_watt_df = pd.DataFrame(cpu_dynamic_energy_watt_consumption)
    cpu_dynamic_watt_df = cpu_dynamic_watt_df.reindex(columns=sorted(cpu_dynamic_watt_df.columns))

    gpu_dynamic_energy_consumption = convert_to_series(gpu_dynamic_energy_consumption)
    gpu_dynamic_df = pd.DataFrame(gpu_dynamic_energy_consumption)
    gpu_dynamic_df = gpu_dynamic_df.reindex(columns=sorted(gpu_dynamic_df.columns))

    idle_cpu_consumption_results = convert_to_series(idle_cpu_consumption_results)
    idle_cpu_df = pd.DataFrame(idle_cpu_consumption_results)
    idle_cpu_df = idle_cpu_df.reindex(columns=sorted(idle_cpu_df.columns))

    temperature_begin = convert_to_series(temperature_begin)
    temperature_begin_df = pd.DataFrame(temperature_begin)
    temperature_begin_df = temperature_begin_df.reindex(columns=sorted(temperature_begin_df.columns))

    temperature_end = convert_to_series(temperature_end)
    temperature_end_df = pd.DataFrame(temperature_end)
    temperature_end_df = temperature_end_df.reindex(columns=sorted(temperature_end_df.columns))




In [37]:
if get_measurements:
    for column in duration_df.columns:
        duration_df[column] = duration_df[column] / 1000

In [38]:
max(list(cpu_df.max()))

394.78668212890625

In [39]:
if get_measurements and show_plots:
    test_boxplot = dram_df.boxplot(figsize=(15,1), rot=75, column=[dram_df.columns[0]])
    test_boxplot.plot()
    plot.ylim(0, 40)
    plot.show()
    
    
    # print("Combined DRAM Energy Consumption in Joules")
    # dram_boxplot = dram_df.boxplot(figsize=(15,5), rot=85)
    # # dram_boxplot = dram_df.boxplot(figsize=(15,5), rot=75, column=['fannkuch-redux.exe.ipg.workstationtwo.all.intel-one-api.win32nt'])
    # dram_boxplot.plot()
    # plot.ylim(0, 40)
    # plot.show()

    print("Combined CPU Energy Consumption in Joules")
    cpu_boxplot = cpu_df.boxplot(figsize=(15,5), rot=85)
    cpu_boxplot.plot()
    cpu_boxplot
    plot.ylim(0,max(list(cpu_df.max())) + 200)
    plot.show()

    # print("Combined GPU Energy Consumption in Joules")
    # gpu_boxplot = gpu_df.boxplot(figsize=(15,5), rot=85)
    # gpu_boxplot.plot()
    # plot.ylim(0,50)
    # plot.show()

    print("Duration of Test Case")
    duration_boxplot = duration_df.boxplot(figsize=(15,5), rot=85)
    duration_boxplot.plot()
    # plot.ylim(0,40)
    plot.show()

In [40]:
cpu_dynamic_watt_df

,nbody.exe.win32nt.workstationone.[0].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[1].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[2].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[3].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[4].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[5].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[6].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[7].intel-one-api.ipg,nbody.exe.win32nt.workstationtwo.[0].intel-one-api.ipg,nbody.exe.win32nt.workstationtwo.[1].intel-one-api.ipg,...,spectral-norm.exe.win32nt.workstationtwo.[0].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[1].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[2].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[3].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[4].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[5].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[6].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[7].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[8].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[9].intel-one-api.ipg
0,3.096849,2.994529,2.759637,2.907749,2.712619,2.790147,3.013944,2.684628,2.058698,2.269735,...,1.690782,1.715228,1.741552,2.344590,1.715726,1.771440,0.542158,0.316963,0.491227,0.436487
1,3.143330,2.864949,2.854363,3.313277,6.008034,5.731237,7.723316,8.486210,2.205596,2.243057,...,2.295800,1.783048,1.727280,1.740762,2.283083,1.748617,0.358349,0.159835,0.315565,0.229582
2,2.998949,3.157229,3.010242,2.852772,2.909713,3.007828,2.901803,2.823574,2.009466,1.939643,...,2.229756,1.858094,1.715950,1.730483,1.686544,1.763734,0.242111,0.165937,0.219712,0.338898
3,3.029702,3.390824,2.890421,2.901924,3.030341,3.198407,2.886439,4.003573,2.117334,2.156253,...,2.273924,1.718793,1.699675,1.724554,2.286725,1.754577,0.497284,0.455144,0.558217,0.433239
4,3.072046,3.217539,2.838912,3.049733,2.837679,2.864207,3.516596,2.874265,2.161074,2.280351,...,2.363517,1.804420,1.744158,1.778540,1.691594,1.813224,0.474972,0.288941,0.332296,0.597062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,5.884820,6.555201,5.923913,5.977213,4.804444,5.879105,5.904038,4.825429,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,5.882220,7.759617,4.781474,5.929213,5.935862,4.808034,5.852502,5.815605,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
409,7.096363,5.955376,5.894818,4.832270,5.877482,5.763300,4.816602,5.838407,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,5.815783,6.583613,5.860743,5.862831,4.700964,5.850885,5.884586,4.732555,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
cpu_dynamic_df

,nbody.exe.win32nt.workstationone.[0].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[1].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[2].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[3].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[4].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[5].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[6].intel-one-api.ipg,nbody.exe.win32nt.workstationone.[7].intel-one-api.ipg,nbody.exe.win32nt.workstationtwo.[0].intel-one-api.ipg,nbody.exe.win32nt.workstationtwo.[1].intel-one-api.ipg,...,spectral-norm.exe.win32nt.workstationtwo.[0].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[1].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[2].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[3].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[4].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[5].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[6].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[7].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[8].intel-one-api.ipg,spectral-norm.exe.win32nt.workstationtwo.[9].intel-one-api.ipg
0,-30.460881,-31.200617,-33.611543,-32.202522,-34.075079,-33.359065,-30.997834,-34.255165,2.665045,2.958329,...,14.108810,13.995115,14.054961,22.454100,13.350329,15.093423,254.313294,241.314541,254.075619,245.029265
1,-29.995681,-32.496942,-32.664207,-28.147111,-1.116825,-3.944512,16.094049,23.873947,4.411572,2.649915,...,22.681955,14.950501,13.854509,13.980350,21.298569,14.770900,243.472707,232.050333,243.719474,232.827681
2,-31.440687,-29.572972,-31.105286,-32.752315,-32.103892,-31.181987,-32.119201,-32.862989,2.079703,-0.857724,...,21.746103,16.007684,13.695373,13.836102,12.941510,14.984529,236.617302,232.410138,238.068458,239.274290
3,-31.132906,-27.236080,-32.303589,-32.260781,-30.897460,-29.275951,-32.272836,-21.039957,3.362196,1.646411,...,22.371964,14.045325,13.466768,13.752907,21.349595,14.855127,251.666785,249.461617,258.025015,244.837716
4,-30.709112,-28.969632,-32.818722,-30.782636,-32.824322,-32.618369,-25.971504,-32.355089,3.882233,3.081054,...,23.641512,15.251568,14.091569,14.510508,13.012256,15.683906,250.350825,239.662338,244.705815,254.498646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,-2.558058,4.420362,-1.966111,-1.506854,-13.154219,-2.465651,-2.098022,-12.805349,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,-2.584086,16.469344,-13.391470,-1.986870,-1.838634,-13.177689,-2.613361,-2.884252,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
409,9.567408,-1.580293,-2.257083,-12.956664,-2.422511,-3.623849,-12.971956,-2.655787,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,-3.249003,4.704596,-2.597865,-2.650710,-14.189153,-2.747889,-2.292533,-13.735905,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
if get_measurements and show_plots:
    # print("Dynamic DRAM Energy Consumption in Watts")
    # dynamic_dram_boxplot = dram_dynamic_df.boxplot(figsize=(15,5), rot=75)
    # dynamic_dram_boxplot.plot()
    # plot.ylim(-0.1, 0.25)
    # plot.show()

    print("Dynamic CPU Energy Consumption in Joiles")
    dynamic_cpu_boxplot = cpu_dynamic_df.boxplot(figsize=(15,5), rot=85)
    dynamic_cpu_boxplot.plot()
    plot.ylim(-100, 100)
    plot.show()
    
    print("Dynamic CPU Energy Consumption in Watts")
    dynamic_watt_cpu_boxplot = cpu_dynamic_watt_df.boxplot(figsize=(15,5), rot=85)
    dynamic_watt_cpu_boxplot.plot()
    plot.ylim(0,50)
    plot.show()
    
    

    # print("Dynamic GPU Energy Consumption in Watts")
    # dynamic_gpu_boxplot = gpu_dynamic_df.boxplot(figsize=(15,5), rot=75)
    # dynamic_gpu_boxplot.plot()
    # plot.ylim(-0.1, 0.25)
    # plot.show()


In [43]:
def list_to_label(lst, mapper):
    if len(lst) == 1:
        return mapper[lst[0]]
    elif len(lst) == 2:
        return f"{mapper[lst[0]]} and {mapper[lst[1]]}"
    else:
        return ", ".join([mapper[x] for x in lst[:-1]]) + f" and {mapper[lst[-1]]}"

def get_caption(data_type, name, c_measuring_instrument, c_duts, c_test_cases, c_compiler):
    caption = f"{name} measurements"

    if not ',' in c_measuring_instrument:
        caption += f" by {c_measuring_instrument}"

    if not ',' in c_duts:
        caption += f" on {c_duts}"
        
    if not ',' in c_test_cases:
        caption += f" for test case(s) {c_test_cases}"
        
    if not ',' in c_compiler:
        caption += f" compiled on {c_compiler}"

    return caption

In [44]:
dut_name_mapper = {
    "workstationone" : "DUT 1",
    "workstationtwo" : "DUT 2"
}

compiler_mapper = {
    "clang" : "Clang",
    "min-gw" : "MinGW",
    "intel-one-api" : "oneAPI",
    "msvc" : "MSVC"
}

test_case_mapper = {
    "fannkuch-redux.exe" : "FR",
    "mandelbrot.exe" : "MB",
    "nbody.exe" : "NB",
    "spectral-norm.exe" : "SN"
}

mi_mapper = {
    "ipg" : "IPG",
    "lhm" : "LHM",
    "clamp" : "Clamp",
    "plug" : "Plug",
    "scaphandre" : "SCAP",
    "scaphandre_isolated" : "SCAPI",
    "rapl" : "RAPL"
}

os_mapper = {
    "unix" : 'LIN',
    "win32nt" : "WIN"
}

In [45]:
box_plot_raw_data = [
    {
        "data" : cpu_df,
        "name" : "CPU",
        "data_type" : "energy consumption",
        "width" : 0.9,
        "xmin" : 0,
        "xmax" : 2000,
        "xlabel" : "Average Energy Consumption (Joules)"
    },
    {
        "data" : cpu_dynamic_df,
        "name" : "CPU",
        "data_type" : "DEC",
        "width" : 0.9,
        "xmin" : 0,
        "xmax" : 1000,
        "xlabel" : "Average DEC (Joules)"
    },
    {
        "data" : cpu_dynamic_watt_df,
        "name" : "CPU",
        "data_type" : "DEC per second",
        "width" : 0.9,
        "xmin" : 0,
        "xmax" : 40,
        "xlabel" : "Average DEC (Watts)"
    },
    # {
    #     "data" : dram_dynamic_df,
    #     "name" : "DRAM",
    #     "data_type" : "dynamic energy consumption",
    #     "width" : 0.9,
    #     "xmin" : -0.1,
    #     "xmax" : 0.1,
    #     "xlabel" : "Average Energy Consumption (Watt)"
    # },
    # {
    #     "data" : dram_df,
    #     "name" : "DRAM",
    #     "data_type" : "energy consumption",
    #     "width" : 0.9,
    #     "xmin" : 0,
    #     "xmax" : 40,
    #     "xlabel" : "Total Energy Consumption (Joules)"
    # },
    # {
    #     "data" : gpu_dynamic_df,
    #     "name" : "GPU",
    #     "data_type" : "dynamic energy consumption",
    #     "width" : 0.9,
    #     "xmin" : -0.1,
    #     "xmax" : 0.1,
    #     "xlabel" : "Average Energy Consumption (Watt)"
    # },
    # {
    #     "data" : gpu_df,
    #     "name" : "GPU",
    #     "data_type" : "energy consumption",
    #     "width" : 0.9,
    #     "xmin" : 0,
    #     "xmax" : 50,
    #     "xlabel" : "Total Energy Consumption (Watt)"
    # },
    {
        "data" : duration_df,
        "name" : "Runtime",
        "data_type" : "duration",
        "width" : 0.9,
        "xmin" : 0,
        "xmax" : 50,
        "xlabel" : "Average Runtime (s)"
    }
]

In [46]:
if generate_graphs and get_measurements:
    for plot_data in box_plot_raw_data:
        print("start")
        data = plot_data["data"]
        name = plot_data["name"]
        data_type = plot_data["data_type"]
        width = plot_data["width"]
        height = 0.1 + (len(data.columns) * 0.02)
        xmin = plot_data["xmin"]
        xmax = plot_data["xmax"]
        xlabel = plot_data["xlabel"]
        
        latex_boxes = tex.get_boxes(data)
        plot.clf()
        # latex_boxes = tex.get_boxes(pd.DataFrame(data['fannkuch-redux.exe.ipg.workstationone.all.intel-one-api.win32nt']))
        
        if name == 'Runtime':
            title = f"The average duration"
        else:
            title = f"The {data_type} of the {name}"
        
        
        ytick = str(list(range(1, len(list(data.columns))+1))).replace('[', '').replace(']', '')
        
        yticklables = tex.get_ytick_labels(
            data, 
            used_test_case_names, 
            used_mi_names, 
            used_dut_name, 
            used_allocated_cores, 
            used_compiler_str, 
            used_os_names,
            dut_name_mapper,
            compiler_mapper,
            test_case_mapper,
            mi_mapper,
            os_mapper
            )
        
        labels = f"{experiment_number}-{experiment_name}-{'-'.join(used_mi_names)}-{'-'.join(used_test_case_names)}-{'-'.join(used_compilers)}-{'-'.join(used_dut_name)}-{name.replace(' ', '_')}-{data_type.replace(' ', '_')}".lower()
        
        c_test_cases = list_to_label(used_test_case_names, test_case_mapper)
        c_compiler = list_to_label(used_compilers, compiler_mapper)
        c_duts = list_to_label(used_dut_name, dut_name_mapper)
        c_measuring_instrument = list_to_label(used_mi_names, mi_mapper)
        
        c_test_cases = c_test_cases.replace(".exe", "")
        
        caption = get_caption(data_type, name, c_measuring_instrument, c_duts, c_test_cases, c_compiler)
        
        latex_box_plot = tex.get_boxplot_template(width, height, xlabel, title, ytick, yticklables, xmin, xmax, latex_boxes, caption, labels)

        box_plot_path = f"{experiment_name}/{'-'.join(used_mi_names)}/{'-'.join(used_test_case_names)}/{'-'.join(used_dut_name)}/{'-'.join(used_compilers)}"
        box_plot_file_name = f"{name}_{data_type.replace(' ', '_')}.tex"
        print(f"generating {box_plot_file_name}")
        # print(data)
        path_util.write_to_path(box_plot_path.replace('.exe', ''), box_plot_file_name, latex_box_plot)
else:
    print("generating plots is disabled")

generating plots is disabled


```latex
\begin{figure}
    \centering
    \begin{tikzpicture}[]
        \pgfplotsset{
            width=WIDTH\textwidth,
            height=WIDHT\textheight
        }
        \begin{axis}[
            xlabel={XLABEL}, 
            title={TITLE}, 
            ytick={YTICK},
        yticklabels={
            LABELS
            },
            xmin=XMIN,xmax=XMAX,
            ]
        
        \addplot+ [boxplot prepared={
        lower whisker=LOW_WHISKER,
        lower quartile=LOW_QUANTILE,
        median=MEDIAN,
        upper quartile=UPPER_QUANTILE,
        upper whisker=UPPER_WHISKER
        }, color = red
        ] coordinates{COORDINATES};
        
        \end{axis}
    \end{tikzpicture}
\caption{CAPTION} \label{fig:LABEL}
\end{figure}
```

In [47]:
# \addplot[mark=none, blue] coordinates {(1000,0) (1000,5)};

In [58]:
list(used_df['spectral-norm.exe.win32nt.workstationtwo.[0].intel-one-api.ipg']).sort()

In [72]:
def remove_n_percent(df, column, percent):
    data_to_order = df[column]

    list(data_to_order).sort()
    data_to_order = [x for x in data_to_order if str(x) != 'nan']
    data_to_order
    percent = 20

    n = len(data_to_order)
    outliers = int(n*percent/100) #may want some rounding logic if n is small
    trimmed_data = data_to_order[outliers: n-outliers]
    
    return trimmed_data

In [69]:
len(data_to_order), len(trimmed_data)

(276, 166)

In [62]:
data_to_order

0      1.690782
1      2.295800
2      2.229756
3      2.273924
4      2.363517
         ...   
407         NaN
408         NaN
409         NaN
410         NaN
411         NaN
Name: spectral-norm.exe.win32nt.workstationtwo.[0].intel-one-api.ipg, Length: 412, dtype: float64

In [ ]:
percent = 20
n = len(used_df['spectral-norm.exe.win32nt.workstationtwo.[2].intel-one-api.ipg'])
outliers = n*percent/100 #may want some rounding logic if n is small
trimmed_data = sorted_data[outliers: n-outliers]

In [73]:
cochrans_data = {
    "" : ["with", "without", "percent", "count"]
}

if get_measurements:
    Z = 1.96
    E = 0.03

    # Z = 2.58
    # E = 0.01

    include_outliers = False

    used_df = cpu_dynamic_watt_df

    for column in used_df.columns:
        used_df_without_outlisers = math_util.exclude_outliers(used_df, column)
        used_df_without_n_percent = remove_n_percent(used_df, column, 20)
        
        current_count = used_df[column].count()
        
        values = used_df_without_outlisers
        n_zero_without = math_util.get_cochran(values, Z, E)
        
        values = used_df_without_n_percent
        n_zero_percent = math_util.get_cochran(values, Z, E)
        
        values = list(used_df[column])
        n_zero = math_util.get_cochran(values, Z, E)
        
        cochrans_data[column] = [round(n_zero, 1), round(n_zero_without, 1), round(n_zero_percent, 1), current_count]

pd.DataFrame(cochrans_data).T

,0,1,2,3
,with,without,percent,count
nbody.exe.win32nt.workstationone.[0].intel-one-api.ipg,5368.9,5162.5,3732.9,412.0
nbody.exe.win32nt.workstationone.[1].intel-one-api.ipg,11771.0,11771.0,9192.7,412.0
nbody.exe.win32nt.workstationone.[2].intel-one-api.ipg,5606.7,5119.2,4392.1,412.0
nbody.exe.win32nt.workstationone.[3].intel-one-api.ipg,5678.2,4678.8,4943.7,412.0
nbody.exe.win32nt.workstationone.[4].intel-one-api.ipg,5019.1,4597.3,3990.5,412.0
nbody.exe.win32nt.workstationone.[5].intel-one-api.ipg,5338.2,5005.2,4363.5,412.0
nbody.exe.win32nt.workstationone.[6].intel-one-api.ipg,5034.7,4622.9,4164.6,412.0
nbody.exe.win32nt.workstationone.[7].intel-one-api.ipg,5507.2,4996.8,4370.9,412.0
nbody.exe.win32nt.workstationtwo.[0].intel-one-api.ipg,11.3,4.3,3.6,276.0
